In [5]:
import tensorflow as tf
import numpy as np

# Define a simple model with an Add layer
class SimpleAddModel(tf.keras.Model):
  def __init__(self, dim_size):
    super(SimpleAddModel, self).__init__()
    self.dim_size = dim_size
    self.add = tf.keras.layers.Add()

  def call(self, inputs):
    return self.add(inputs)


# Define the dimension size
dim_size = 5

# Create an instance of the model
model = SimpleAddModel(dim_size)

# Define some dummy input data
input_data = [np.random.rand(1, dim_size).astype(np.float32), np.random.rand(1, dim_size).astype(np.float32)]

# Build the model by calling it
model(input_data)

# Save the model to a SavedModel format
saved_model_dir = "saved_model"
tf.saved_model.save(model, saved_model_dir)

# Convert the model to TFLite with INT8 quantization
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
  for _ in range(100):
    yield [np.random.rand(1, dim_size).astype(np.float32), np.random.rand(1, dim_size).astype(np.float32)]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

file_name = "models/add/model_int8.tflite"

# Save the TFLite model to a file
with open(file_name, "wb") as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: saved_model/assets


2025-02-20 11:09:09.338926: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2025-02-20 11:09:09.338946: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2025-02-20 11:09:09.339038: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: saved_model
2025-02-20 11:09:09.339234: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-02-20 11:09:09.339248: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: saved_model
2025-02-20 11:09:09.339685: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2025-02-20 11:09:09.346363: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: saved_model
2025-02-20 11:09:09.348376: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status: success: OK. Took 9338 microseconds.
ful